In [22]:
import pandas as pd
import os
import matplotlib as mpl
import matplotlib.pyplot as plt

In [23]:

def read_data(file_name: str, hypothesis_flag) -> pd.DataFrame:
    """
    Load file containg details of LCA and retain only useful columns.
    Replace the column names with meaningful heads.
    :param filename: The path to the LCA data file.
    """

    col_required = ['STATUS', 'CASE_STATUS', 'LCA_CASE_EMPLOYER_NAME', 'EMPLOYER_NAME', 'TOTAL_WORKERS',
                    'TOTAL_WORKER_POSITIONS',
                    'LCA_CASE_WORKLOC1_STATE', 'VISA_CLASS', 'LCA_CASE_NUMBER', 'LCA_CASE_NAICS_CODE', 'NAICS_CODE',
                    'TOTAL WORKERS', 'WORKSITE_STATE', 'WORKSITE_STATE_1', 'CASE_NUMBER', 'NAIC_CODE']

    df = pd.read_csv(filepath_or_buffer=file_name, usecols=lambda x: x in col_required,
                     dtype={'LCA_CASE_NAICS_CODE': 'str',
                            'NAICS_CODE': 'str',
                            'NAIC_CODE': 'str',
                            },
                     low_memory=False, encoding='ISO-8859-1')

    df = df.rename(
        columns={'LCA_CASE_NUMBER': 'CASE_NUMBER', 'CASE_STATUS': 'STATUS', 'LCA_CASE_EMPLOYER_NAME': 'EMPLOYER_NAME',
                 'TOTAL_WORKER_POSITIONS': 'TOTAL_WORKERS', 'LCA_CASE_WORKLOC1_STATE': 'WORKSITE_STATE',
                 'LCA_CASE_NAICS_CODE': 'NAICS_CODE', 'WORKSITE_STATE_1': 'WORKSITE_STATE', 'NAIC_CODE': 'NAICS_CODE',
                 'TOTAL WORKERS': 'TOTAL_WORKERS'})
    if hypothesis_flag == False:
            df['STATUS'] = df["STATUS"].str.upper()
            df = df[(df['STATUS'] == 'CERTIFIED') & (df['VISA_CLASS'] == 'H-1B')]
            df['NAICS_CODE'] = df['NAICS_CODE'].str[:2]
            return (df)
    elif hypothesis_flag == True:
            df['STATUS'] = df["STATUS"].str.upper()            
            df['NAICS_CODE'] = df['NAICS_CODE'].str[:2]
            df = df[df['VISA_CLASS'] == 'H-1B']
            return (df)

In [24]:
def sector_range(row):
    if isinstance(row, list) and len(row) > 1:
        return list(range(int(row[0]), int(row[1]) + 1))
    elif isinstance(row, list):
        return row[0]

def read_sector_data (filename : str) -> pd.DataFrame:
    """
    Load the NAICS data file, retaining only the most useful columns & rows.
    Change the layout for a few rows to make the data inclusive of a codes.
    :param filename: The path to the NAICS code data file.
 
    """
    sector_df = pd.read_csv(filepath_or_buffer=filename,encoding='ISO-8859-1')
    sector_df["Sector"]=sector_df["Sector"].str.split("-")
    sector_df['sector_range'] = sector_df["Sector"].apply(sector_range)
    sector_codes = sector_df['sector_range'].apply(pd.Series).reset_index().melt(id_vars='index').dropna()[['index', 'value']].set_index('index')
    sector_codes_final = sector_codes.merge(sector_df['Name'], left_index=True, right_index=True, how='inner')
    sector_codes_final = sector_codes_final.rename(columns={"value": "Sector"})
    sector_codes_final['Sector']= sector_codes_final['Sector'].astype('int8')
    return(sector_codes_final)

def hypothesis_one_cal(year_df, sector_data_df, yy):
    year_df =year_df.merge(sector_data_df, how='left', left_on='NAICS_CODE', right_on='Sector')
    stats_df = year_df.groupby(['Name'])['TOTAL_WORKERS'].sum().astype('int32').reset_index(name=yy)
    return( stats_df)

def hypothesis_one(file_list):
    sector_df = read_sector_data ('2017_NAICS_Structure_Summary_Table.csv')
    sector_df['Sector'] = sector_df['Sector'].astype('str')
    sector_name = sector_df.Name.unique().tolist()
    plot_data_df = pd.DataFrame()
    plot_data_df['Sectors']= sector_name
    for file in file_list:
        file_name = "data_H1B/" + file
        hypothesis_flag = False
        year_data = read_data (file_name, hypothesis_flag)
        year = '20' + file[7:9]
        stats = hypothesis_one_cal(year_data,sector_df,year)
        plot_data_df = plot_data_df.merge(stats, how='left', left_on='Sectors', right_on='Name')
        del plot_data_df['Name']
    return(plot_data_df)

In [25]:
def hypothesis_two(directory):
    country = ['China - mainland', 'China - Taiwan','India', 'Korea, South', 'Mexico','Brazil', 'Australia', 'Russia','Great Britain and Northern Ireland',
                'Germany','France', 'Philippines']
    sd= pd.DataFrame({"Nationality": country})
    for file in directory:
        col_name = 'Fiscal Year 20' + file[2:4]
        file_name = "data_Country/" + file
        df = pd.read_csv(filepath_or_buffer=file_name,thousands=',', dtype= {'H-1B':'float'})
        df=df.rename(columns={'Unnamed: 0': 'Visa_Country', col_name: 'Visa_Country'})
        df= df[['Visa_Country','H-1B']]
        df= df.rename(columns ={'H-1B': col_name})
        sd =sd.merge(df, how='left',  left_on= 'Nationality', right_on='Visa_Country')
        del sd['Visa_Country']
    plot_hypothesis_two(sd)
    return (sd)
def plot_hypothesis_two(data_plot):
    data_plot = data_plot.set_index('Nationality')
    data_plot = data_plot.T
    data_plot = data_plot.pct_change()
    data_plot.plot.barh(legend = False)

In [26]:
def df_hypothesis_three(file_list):
    company_df = pd.read_csv("companylist.csv", dtype={'MarketCap': 'float64'})
    final_df = pd.DataFrame()
    list_of_df = []
    for file in file_list:
        file_name = "data_H1B/" + file
        hypothesis_flag = True
        year_data = read_data (file_name, hypothesis_flag)
        year = '20' + file[7:9]
        company_df['name_lower'] = company_df['Name'].str.lower()
        year_data['EMPLOYER_NAME_lower'] = year_data['EMPLOYER_NAME'].str.lower()
        companylist_merged_df = year_data.merge(company_df, left_on='EMPLOYER_NAME_lower', right_on='name_lower')
        companylist_merged_df_certified = pd.DataFrame()
        companylist_merged_df_total = pd.DataFrame()
        companylist_merged_df_total = companylist_merged_df.groupby(['EMPLOYER_NAME','MarketCap'])['TOTAL_WORKERS'].sum().reset_index(name="TOTAL_WORKERS_OVERALL")
        companylist_merged_df_certified = companylist_merged_df[companylist_merged_df['STATUS'] == 'CERTIFIED'].groupby(['EMPLOYER_NAME','MarketCap'])['TOTAL_WORKERS'].sum().reset_index(name="CERTIFIED_TOTAL_WORKERS")
        companylist_merged_df_certified['year'] = year
        companylist_merged_df_total['year'] = year
        new_data = companylist_merged_df_certified.merge(companylist_merged_df_total, on='EMPLOYER_NAME')  
        new_data['MarketCap_x'].astype('float64')
        new_data['Rate'] = (new_data['CERTIFIED_TOTAL_WORKERS'] / new_data['TOTAL_WORKERS_OVERALL'])*100
        new_data = new_data[['EMPLOYER_NAME', 'Rate', 'MarketCap_x','year_x']]
        #print(new_data.sort_values(by='Rate', ascending=False))
        list_of_df.append(new_data)
    final_df = pd.concat(list_of_df,ignore_index=True)
    return final_df   

In [27]:
def hypothesis_two_state_cal(states_data_df, year_df, yy):
    year_df =year_df.merge(states_data_df, how='left', left_on='WORKSITE_STATE', right_on='Abbreviation')
    stats_df = year_df.groupby(['State'])['TOTAL_WORKERS'].sum().reset_index(name=yy)
    return( stats_df)

def hypothesis_two_state(file_list):
    states_df = pd.read_csv('states.csv')
    state_name = states_df.State.unique().tolist()
    plot_data_df = pd.DataFrame()
    plot_data_df['Worksite State']= state_name
    for file in file_list:
        file_name = "data_H1B/" + file
        hypothesis_flag = False
        year_data = read_data (file_name, hypothesis_flag)
        year = '20' + file[7:9]
        stats = hypothesis_two_state_cal(states_df,year_data, year)
        plot_data_df = plot_data_df.merge(stats, how='left', left_on='Worksite State', right_on='State')
        del plot_data_df['State']
    plot_data_df = plot_data_df.set_index('Worksite State')
    plot_data_df = plot_data_df.apply(lambda s: pd.Series(s.nlargest(5).index))
    
    return(plot_data_df)

In [28]:

if __name__ == '__main__':
    path = "data_H1B/"
    directory = os.listdir(path)
    path2 = "data_Country/"
    directory2 =os.listdir(path2)
    df = hypothesis_one(directory)
    sd= hypothesis_two(directory2)
    df = hypothesis_two_state(directory)
    df = hypothesis_one(directory)
    hypothesis3_df=df_hypothesis_three(directory)   

In [29]:
df

,Sectors,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,"Agriculture, Forestry, Fishing and Hunting",452,457,401,393,462,415,426,531,281,212
1,"Mining, Quarrying, and Oil and Gas Extraction",1464,1669,1607,1808,1635,1227,1225,1201,1085,1023
2,Utilities,971,983,881,1023,991,972,1108,1098,1226,1120
3,Construction,1944,2008,1798,1914,2431,2546,2792,3324,3718,3056
4,Manufacturing,37772,38182,48551,57489,58325,80784,122594,146438,154359,152264
5,Wholesale Trade,4829,5025,5634,5953,7161,7253,7567,7268,7030,5411
6,Retail Trade,7660,10274,10895,12841,17770,16378,22453,26537,28070,32169
7,Transportation and Warehousing,1586,1691,1486,1734,1889,1938,2315,2634,2612,2396
8,Information,25867,32893,22842,28226,36460,43883,50057,56962,62731,63067
9,Finance and Insurance,17025,15949,17371,22244,25191,32069,34703,38859,39831,40386


In [78]:
hypothesis3_df = hypothesis3_df[hypothesis3_df.MarketCap_x != 0.000000e+00] 


In [99]:
head_df = pd.DataFrame()
for year in range(2011, 2021):      
    head_df = head_df.append(hypothesis3_df[hypothesis3_df.year_x == str(year)].sort_values(by='MarketCap_x', ascending=False).head(3))
head_df

,EMPLOYER_NAME,Rate,MarketCap_x,year_x
236,MICROSOFT CORPORATION,88.883732,8.299033e+11,2011
30,APPLE INC.,94.230769,8.059828e+11,2011
147,"FACEBOOK, INC.",85.777778,4.631740e+11,2011
710,MICROSOFT CORPORATION,91.985089,8.299033e+11,2012
413,APPLE INC.,96.645085,8.059828e+11,2012
624,"FACEBOOK, INC.",88.461538,4.631740e+11,2012
1083,MICROSOFT CORPORATION,85.682417,8.299033e+11,2013
882,APPLE INC.,81.610537,8.059828e+11,2013
985,"FACEBOOK, INC.",94.053398,4.631740e+11,2013
1509,MICROSOFT CORPORATION,76.411048,8.299033e+11,2014


In [101]:
tail_df = pd.DataFrame()
for year in range(2011, 2021):          
    tail_df = tail_df.append(hypothesis3_df[hypothesis3_df.year_x == str(year)].sort_values(by='MarketCap_x', ascending=False).tail(3))
tail_df

,EMPLOYER_NAME,Rate,MarketCap_x,year_x
56,"BROADVISION, INC.",50.000000,7.848672e+06,2011
295,"PULMATRIX, INC.",100.000000,6.004046e+06,2011
54,"BRIDGELINE DIGITAL, INC.",100.000000,3.462306e+06,2011
491,CREDIT SUISSE AG,100.000000,5.240500e+06,2012
493,CREDIT SUISSE AG,100.000000,5.240500e+06,2012
445,"BRIDGELINE DIGITAL, INC.",60.000000,3.462306e+06,2012
892,"AVINGER, INC.",66.666667,1.081421e+07,2013
1091,"MOSYS, INC.",100.000000,8.519048e+06,2013
903,"BRIDGELINE DIGITAL, INC.",100.000000,3.462306e+06,2013
1308,"BROADVISION, INC.",100.000000,7.848672e+06,2014
